In [3]:

# 先颜色后形状
import cv2
import numpy as np
from collections import Counter
import time 	


def process(path):
     # 根据路径读取一张图片
    start=time.clock()    
    image = cv2.imread(path)
#     显示原图
    cv2.imshow('origin',image)
    shapes=image.shape
    # 剪切有效区域 ：判断图片的大小，是否需要裁剪
    images_size=[shapes[0],shapes[1]]
    images_size=np.min(images_size)
    if images_size > 1000:
        image=image[0:int(1/3*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
    elif images_size > 800:
        image=image[int(1/6*shapes[1]):int(5/6*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
    else:
        image=image
#         保存剪切图片
#     cv2.imwrite('cut.jpg',image)
    # 计算有效区域的面积
    shapes=image.shape
    propotion=image.shape[0]*image.shape[1]
     # BGR转HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     cv2.imshow('hsv',hsv)
    #保留底片
    origin_copy=image.copy()
    origin_gray=cv2.cvtColor(origin_copy,cv2.COLOR_BGR2GRAY)
    return hsv,propotion,shapes,origin_copy,origin_gray


# 绿色的范围
def getGreen(image):    
    lower_green = np.array([45, 70, 100])
    upper_green = np.array([85, 255, 255])
#     cv2.imshow('green',getRange(image,lower_green,upper_green))
    return {'绿色':getRange(image,lower_green,upper_green)}

# 红色的范围 有两个位置
def getRed(image):    
    f_low_red = np.array([0, 190, 200])
    f_high_red = np.array([4, 255, 255])
    low_red = np.array([170, 120, 130])
    high_red = np.array([180, 255, 255])
    img1=getRange(image,f_low_red,f_high_red)
    img2=getRange(image,low_red,high_red)
    cv2.imshow('red',img1+img2)
    return {'红色':img1+img2}

# 黄色的范围
def getYellow(image):    
    low_yellow = np.array([6, 100, 220])
    high_yellow = np.array([45, 255, 255])
#     cv2.imshow('yellow',getRange(image,low_yellow,high_yellow))
    return {'黄色':getRange(image,low_yellow,high_yellow)}

def getRange(image,low,high):
# cv2.inRange函数设阈值，去除背景部分
    mask = cv2.inRange(image, low, high)
    return mask

def deNoise(img):  
#     cv2.imshow('img',img)
    erode=cv2.erode(img,(100,100),iterations=4)
    cv2.imshow('erode',erode)

#降噪（模糊处理用来减少瑕疵点）
    blur = cv2.blur(erode, (5,5))   
    cv2.imshow('blur',blur)

# 膨胀
    dilate=cv2.dilate(blur,(100,100),iterations=4)
    cv2.imshow('dilate',dilate)

# 边缘检测    
    canny = cv2.Canny(dilate, 150, 240)  
    cv2.imshow('canny',canny)

# 闭操作   
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10)) 
    closed = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)
    cv2.imshow('closed',closed)

    return closed

#求重合区域
def overlap(a,b):
    flag=False
    overlapThresh=0.4
    area = (a[2] - a[0] + 1) * (a[3] - a[1] + 1)  
    xx1 = max(a[0], b[0])  
    yy1 = max(a[1], b[1])  
    xx2 = min(a[2], b[2])  
    yy2 = min(a[3], b[3])  
    w = max(0, xx2 - xx1 + 1)  
    h = max(0, yy2 - yy1 + 1)  
    overlap = float(w * h) / area  
    if overlap > overlapThresh:  
        flag=True
    return flag
#非最大值抑制,减少重叠区域
def non_max_suppression_slow(boxes, overlapThresh):  
    if len(boxes) == 0:  
        return []  
   
    pick = [] 
    x1 = boxes[:,0]  
    y1 = boxes[:,1]  
    x2 = boxes[:,2]  
    y2 = boxes[:,3]  
   
    area = (x2 - x1 + 1) * (y2 - y1 + 1)  
    idxs = np.argsort(y2)
    while len(idxs) > 0:  
        last = len(idxs) - 1  
        i = idxs[last]  
        pick.append(i)  
        suppress = [last]  
        for pos in range(0, last):  
            j = idxs[pos]  
            xx1 = max(x1[i], x1[j])  
            yy1 = max(y1[i], y1[j])  
            xx2 = min(x2[i], x2[j])  
            yy2 = min(y2[i], y2[j])  
            w = max(0, xx2 - xx1 + 1)  
            h = max(0, yy2 - yy1 + 1)  
            overlap = float(w * h) / area[j]  
            if overlap > overlapThresh:  
                suppress.append(pos)  
        idxs = np.delete(idxs, suppress)  
    return boxes[pick]  
    
#获取最终ROI的坐标    
def getPositions(img,propotion,shapes,origin_copy,origin_gray):    
    cv2.imshow('img',img)
    ret,thresh = cv2.threshold(img,127,255,0)
    tmp,contours,hierarchy=cv2.findContours(thresh,1,2)    
    hsvList=[]
    area_list=[]
#有效交通灯的比例    
    if contours :
        for i in range(len(contours)):
            x,y,w,h=cv2.boundingRect(contours[i])
            cv2.rectangle(origin_copy, (x, y), (x+w, y+h), (0, 255, 0), 2) 
            if cv2.contourArea(contours[i])/propotion >0.0002 and cv2.contourArea(contours[i])/propotion < 0.1:
# 最小面积矩形                
                x,y,w,h=cv2.boundingRect(contours[i])
                y0=int(y-0.2*h)
                y1=int(y+0.9*h)
                x0=int(x-0.1*w)
                x1=int(x+w)
                if y0 <0:
                    y0=0
                if y1>shapes[0]:
                    y1=shapes[0]
                if x0<0:
                    x0=0
                if x1>shapes[1]:
                    x1=shapes[1]                
                area_list.append([x0,y0,x1,y1])
#                 cv2.rectangle(origin_copy, (x, y), (x1, y1), (0, 255, 0), 2) 
        area_list=np.array(area_list)         
        cv2.imshow('propotion',origin_copy)

# 非极大值抑制   
        prop=non_max_suppression_slow(area_list,0.7)
        if len(prop):
            i=0
            shape=[]
            for (startX, startY, endX, endY) in prop:                 
#算出每个最小区域的亮度，进行从大到小的排序，将最亮的排在第一位
                HSV=origin_copy[startY:endY,startX:endX,:]
                HSV = cv2.cvtColor(HSV, cv2.COLOR_BGR2HSV)        
                H, S, V = cv2.split(HSV)
                v = np.mean(V)
                hsvList.append([i,v,[y0,y1,x0,x1]])
                
#                 基于灰度的模板匹配法
                methods = ['cv2.TM_CCOEFF', 'cv2.TM_SQDIFF']
                templates=['test.png','test2.png','test3.png','test4.png']
                for template in templates:  
                    template_name=template
                    template=cv2.imread(template,0)
                    w, h = template.shape[:2]
                    for meth in methods:
                        method = eval(meth)
                        res = cv2.matchTemplate(origin_gray,template,method)  
                        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)  
                        if method in [cv2.TM_SQDIFF]:  
                            top_left = min_loc  

                        else:  
                            top_left = max_loc  
                        bottom_right = (top_left[0] + w, top_left[1] + h)
                        if overlap([startX,startY,endX,endY],[top_left[0],top_left[1],bottom_right[0],bottom_right[1]]):
                            if(template_name=='test.png'):
                                shape.append('圆形5米')
                            elif(template_name=='test2.png'):
                                shape.append("T型15米")
                            elif(template_name=='test3.png'):
                                shape.append("左箭头5米")
                            elif(template_name=='test4.png'):
                                shape.append("上箭头5米")
#                             else:
#                                 shape.append("上箭头")
                            cv2.rectangle(origin_copy,top_left, bottom_right, 255, 2)                
                cv2.rectangle(origin_copy, (startX, startY), (endX, endY), (0, 255, 0), 2) 
                i+=1 
            if shape:
                shapes=Counter(shape)
                shapes=shapes.most_common()[0]
                shape.append(shapes[0])
            else:
                shape.append('')
            hsvList=sorted(hsvList,key=lambda x : (x[1]),reverse=True) 
            order=hsvList[0][0]
            target=order
#             cv2.putText(origin_copy, "IoU", (target[0], target[1]),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.imshow('res',origin_copy)
            return shape[0],hsvList[0][2]
        else:
            return '无形状',0
    else:
        return '无形状',0
    

def main(path):
    start=time.clock()
    hsv,propotion,shapes,origin_copy,origin_gray=process(path)    
    colors=[]
#     colors.append(getGreen(hsv))
    colors.append(getRed(hsv))
#     colors.append(getYellow(hsv))
    # 节省时间复杂度

    areas=[]
    sumResult=0
    for i in range(len(colors)):
        closed = deNoise(list(colors[i].values())[0])        
        shape,coordinate = getPositions(closed,propotion,shapes,origin_copy,origin_gray)
        if isinstance(coordinate,list):
            print('coordinate',coordinate)
            HSV=origin_copy[coordinate[0]:coordinate[1],coordinate[2]:coordinate[3],:]
            HSV = cv2.cvtColor(HSV, cv2.COLOR_BGR2HSV)        
            H, S, V = cv2.split(HSV)
            v = np.mean(V)
            areas.append((list(colors[i].keys())[0],v,shape))
        else:
            areas.append((list(colors[i].keys())[0],0,shape))

    # 按颜色值的大小排序
    areas=sorted(areas,key=lambda x : (x[1]),reverse=True)
    print(areas)
    ended=time.clock()
    print('total:',ended-start)
    if int(areas[0][1]) != 0:
        print('The current color is:',areas[0][0],areas[0][2])
    else:
        print('没有交通灯')
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__=='__main__':
    path="light7.jpg"
    main(path)

coordinate [58, 83, 803, 822]
[('红色', 202.70315789473685, 'T型15米')]
total: 1.8945260659420455
The current color is: 红色 T型15米


In [9]:
# 形状与颜色相与
import cv2
import numpy as np
import time 	

def getShapes(origin_copy):
#                 基于灰度的模板匹配法
    origin_copy=cv2.cvtColor(origin_copy,cv2.COLOR_BGR2GRAY)
    ans=[]
    methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCORR', 'cv2.TM_SQDIFF']
    templates=['test.png','test2.png','test3.png','test4.png']
    for template in templates:
        template_name=template
        template=cv2.imread(template,0)
        w, h = template.shape[:2]
        for meth in methods:
            method = eval(meth)
            res = cv2.matchTemplate(origin_copy,template,method)  
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)  
            if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:  
                top_left = min_loc
            else:  
                top_left = max_loc  
            bottom_right = (top_left[0] + w, top_left[1] + h)        
            ans.append([template_name,[top_left[0],top_left[1],bottom_right[0],bottom_right[1]]])
#     ans=np.array(ans)
#     print('ans',ans)
    return ans
    
def process(path):
     # 根据路径读取一张图片
    start=time.clock()    
    image = cv2.imread(path)
    shapes=image.shape
    # 剪切有效区域 ：判断图片的大小，是否需要裁剪
    images_size=[shapes[0],shapes[1]]
    images_size=np.min(images_size)
    if images_size > 1000:
        image=image[0:int(1/3*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
    elif images_size > 800:
        image=image[int(1/6*shapes[1]):int(5/6*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
    else:
        image=image
#     cv2.imshow('image',image)
    # 计算有效区域的面积
    shapes=image.shape
    propotion=image.shape[0]*image.shape[1]
     # BGR转HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    #保留底片
    origin_copy=image.copy()
    return hsv,propotion,shapes,origin_copy


# 绿色的范围
def getGreen(image):    
    lower_green = np.array([45, 70, 100])
    upper_green = np.array([85, 255, 255])
    return {'green':getRange(image,lower_green,upper_green)}

# 红色的范围
def getRed(image):    
#     low_red = np.array([0, 100, 80])
#     high_red = np.array([8, 255, 255])
    low_red = np.array([170, 120, 130])
    high_red = np.array([255, 255, 255])
    return {'red':getRange(image,low_red,high_red)}

# 绿色的范围
def getYellow(image):    
#     low_yellow = np.array([10, 100, 220])
    low_yellow = np.array([0, 100, 220])
    high_yellow = np.array([45, 255, 255])
    return {'yellow':getRange(image,low_yellow,high_yellow)}

def getRange(image,low,high):
# cv2.inRange函数设阈值，去除背景部分
    mask = cv2.inRange(image, low, high)
    return mask

def deNoise(img):   
#     cv2.imshow('img',img)
    erode=cv2.erode(img,(100,100),iterations=4)
#     cv2.imshow('erode',img)
#降噪（模糊处理用来减少瑕疵点）
    blur = cv2.blur(erode, (5,5))   
# 膨胀
    dilate=cv2.dilate(blur,(100,100),iterations=4)
# 边缘检测    
    canny = cv2.Canny(dilate, 150, 240)     
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
# 闭操作    
    closed = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)
#     cv2.imshow('closed',closed)
    return closed
#求重合区域
def overlap(a,b):
#     [x0,y0,x1,y1]
    flag=False
    overlapThresh=0.6
    area = (a[2] - a[0] + 1) * (a[3] - a[1] + 1)  
    xx1 = max(a[0], a[0])  
    yy1 = max(a[1], b[1])  
    xx2 = min(a[2], b[2])  
    yy2 = min(a[3], b[3])  
    w = max(0, xx2 - xx1 + 1)  
    h = max(0, yy2 - yy1 + 1)  
    overlap = float(w * h) / area  
    if overlap > overlapThresh:  
        flag=True
    return flag
#非最大值抑制,减少重叠区域
def non_max_suppression_slow(boxes, overlapThresh):  
    if len(boxes) == 0:  
        return []  
   
    pick = [] 
    x1 = boxes[:,0]  
    y1 = boxes[:,1]  
    x2 = boxes[:,2]  
    y2 = boxes[:,3]  
   
    area = (x2 - x1 + 1) * (y2 - y1 + 1)  
    idxs = np.argsort(y2)
    while len(idxs) > 0:  
        last = len(idxs) - 1  
        i = idxs[last]  
        pick.append(i)  
        suppress = [last]  
        for pos in range(0, last):  
            j = idxs[pos]  
            xx1 = max(x1[i], x1[j])  
            yy1 = max(y1[i], y1[j])  
            xx2 = min(x2[i], x2[j])  
            yy2 = min(y2[i], y2[j])  
            w = max(0, xx2 - xx1 + 1)  
            h = max(0, yy2 - yy1 + 1)  
            overlap = float(w * h) / area[j]  
            if overlap > overlapThresh:  
                suppress.append(pos)  
        idxs = np.delete(idxs, suppress)  
    return boxes[pick]  
    
#获取最终ROI的坐标    
def getPositions(img,propotion,shapes,origin_copy):    
    ret,thresh = cv2.threshold(img,127,255,0)
    tmp,contours,hierarchy=cv2.findContours(thresh,1,2)    
    hsvList=[]
    area_list=[]
#有效交通灯的比例    
    if contours :
        for i in range(len(contours)):
            if cv2.contourArea(contours[i])/propotion >0.0002 and cv2.contourArea(contours[i])/propotion < 0.1:
# 最小面积矩形                
                x,y,w,h=cv2.boundingRect(contours[i])                
                y0=int(y-0.2*h)
                y1=int(y+0.9*h)
                x0=int(x-0.1*w)
                x1=int(x+w)
                if y0 <0:
                    y0=0
                if y1>shapes[0]:
                    y1=shapes[0]
                if x0<0:
                    x0=0
                if x1>shapes[1]:
                    x1=shapes[1]                
                area_list.append([x0,y0,x1,y1])
        area_list=np.array(area_list)         
        
# 非极大值抑制   
        prop=non_max_suppression_slow(area_list,0.7)
        if len(prop):
            i=0
            for (startX, startY, endX, endY) in prop:                 
#算出每个最小区域的亮度，进行从大到小的排序，将最亮的排在第一位
                HSV=origin_copy[startY:endY,startX:endX,:]
                HSV = cv2.cvtColor(HSV, cv2.COLOR_BGR2HSV)        
                H, S, V = cv2.split(HSV)
                v = np.mean(V)
                hsvList.append([i,v,[y0,y1,x0,x1]])
#                 形状模板匹配
                ans=getShapes(origin_copy)
                for i in range(len(ans)):
                    template_name=ans[i][0]
                    pos=ans[i][1]
                    if(overlap([startX,startY,endX,endY],pos)):
                        if(template_name=='test.png'):
                            print('圆形')
                        elif(template_name=='test2.png'):
                            print("T型")
                        elif(template_name=='test3.png'):
                            print("左箭头")
                        elif(template_name=='test4.png'):
                            print("上箭头")
                        else:
                            print('右箭头')
#                         cv2.rectangle(origin_copy,top_left, bottom_right, 255, 2)
                        cv2.rectangle(origin_copy,(pos[0],pos[1]), (pos[2],pos[3]), 255, 2)
                cv2.rectangle(origin_copy, (startX, startY), (endX, endY), (0, 255, 0), 2) 
                i+=1 
            hsvList=sorted(hsvList,key=lambda x : (x[1]),reverse=True) 
            order=hsvList[0][0]
            target=order
#             cv2.putText(origin_copy, "IoU", (target[0], target[1]),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.imshow('res',origin_copy)
            return hsvList[0][2]
        else:
            return 0
    else:
        return 0
    

def main(path):
    start=time.clock()
    hsv,propotion,shapes,origin_copy=process(path)    
    colors=[]
#     colors.append(getGreen(hsv))
#     colors.append(getRed(hsv))
    colors.append(getYellow(hsv))
    # 节省时间复杂度

    areas=[]
    sumResult=0
    for i in range(len(colors)):
        closed = deNoise(list(colors[i].values())[0])
        coordinate = getPositions(closed,propotion,shapes,origin_copy)
        if isinstance(coordinate,list):
            print('coordinate',coordinate)
            HSV=origin_copy[coordinate[0]:coordinate[1],coordinate[2]:coordinate[3],:]
            HSV = cv2.cvtColor(HSV, cv2.COLOR_BGR2HSV)        
            H, S, V = cv2.split(HSV)
            v = np.mean(V)
            areas.append((list(colors[i].keys())[0],v))
        else:
            areas.append((list(colors[i].keys())[0],0))

    # 按颜色值的大小排序
    areas=sorted(areas,key=lambda x : (x[1]),reverse=True)
    print(areas)
    ended=time.clock()
    print('total:',ended-start)
    if int(areas[0][1]) != 0:
        print('The current color is:',areas[0][0])
    else:
        print('没有交通灯')
    cv2.waitKey(0)
    cv2.destroyAllWindows()


if __name__=='__main__':
    path="left0588.jpg"
    main(path)

左箭头
上箭头
圆形
左箭头
上箭头
圆形
T型
左箭头
上箭头
coordinate [324, 348, 85, 107]
[('yellow', 234.79545454545453)]
total: 0.7442637199219462
The current color is: yellow


In [ ]:
# 形状与颜色相与
import cv2
import numpy as np
import time 	

def getShapes(origin_copy):
#                 基于灰度的模板匹配法
    ans=[]
    methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCORR', 'cv2.TM_SQDIFF']
    templates=['test.png','test2.png','test3.png','test4.png']
    for template in templates:
        template_name=template
        template=cv2.imread(template)
        w, h = template.shape[:2]
        for meth in methods:
            method = eval(meth)
            res = cv2.matchTemplate(origin_copy,template,method)  
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)  
            if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:  
                top_left = min_loc
            else:  
                top_left = max_loc  
            bottom_right = (top_left[0] + w, top_left[1] + h)        
            ans.append([template_name,[top_left[0],top_left[1],bottom_right[0],bottom_right[1]]])
#     ans=np.array(ans)
#     print('ans',ans)
    return ans
    
def process(path):
     # 根据路径读取一张图片
    start=time.clock()    
    image = cv2.imread(path)
    shapes=image.shape
    # 剪切有效区域 ：判断图片的大小，是否需要裁剪
    images_size=[shapes[0],shapes[1]]
    images_size=np.min(images_size)
    if images_size > 1000:
        image=image[0:int(1/3*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
    elif images_size > 800:
        image=image[int(1/6*shapes[1]):int(5/6*shapes[1]),int(1/12*shapes[0]):int(11/12*shapes[1]),:]
    else:
        image=image
#     cv2.imshow('image',image)
    # 计算有效区域的面积
    shapes=image.shape
    propotion=image.shape[0]*image.shape[1]
     # BGR转HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    #保留底片
    origin_copy=image.copy()
    return hsv,propotion,shapes,origin_copy


# 绿色的范围
def getGreen(image):    
    lower_green = np.array([45, 70, 100])
    upper_green = np.array([85, 255, 255])
    return {'green':getRange(image,lower_green,upper_green)}

# 红色的范围
def getRed(image):    
    low_red = np.array([0, 100, 80])
    high_red = np.array([8, 255, 255])
    return {'red':getRange(image,low_red,high_red)}

# 黄色的范围
def getYellow(image):    
    low_yellow = np.array([10, 100, 220])
    high_yellow = np.array([45, 255, 255])
    return {'yellow':getRange(image,low_yellow,high_yellow)}

def getRange(image,low,high):
# cv2.inRange函数设阈值，去除背景部分
    mask = cv2.inRange(image, low, high)
    return mask

def deNoise(img):   
    erode=cv2.erode(img,(100,100),iterations=4)
#降噪（模糊处理用来减少瑕疵点）
    blur = cv2.blur(erode, (5,5))   
# 膨胀
    dilate=cv2.dilate(blur,(100,100),iterations=4)
# 边缘检测    
    canny = cv2.Canny(dilate, 150, 240)     
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
# 闭操作    
    closed = cv2.morphologyEx(canny, cv2.MORPH_CLOSE, kernel)
#     cv2.imshow('closed',closed)
    return closed
#求重合区域
def overlap(a,b):
#     [x0,y0,x1,y1]
    flag=False
    overlapThresh=0.6
    area = (a[2] - a[0] + 1) * (a[3] - a[1] + 1)  
    xx1 = max(a[0], a[0])  
    yy1 = max(a[1], b[1])  
    xx2 = min(a[2], b[2])  
    yy2 = min(a[3], b[3])  
    w = max(0, xx2 - xx1 + 1)  
    h = max(0, yy2 - yy1 + 1)  
    overlap = float(w * h) / area  
    if overlap > overlapThresh:  
        flag=True
    return flag
#非最大值抑制,减少重叠区域
def non_max_suppression_slow(boxes, overlapThresh):  
    if len(boxes) == 0:  
        return []  
   
    pick = [] 
    x1 = boxes[:,0]  
    y1 = boxes[:,1]  
    x2 = boxes[:,2]  
    y2 = boxes[:,3]  
   
    area = (x2 - x1 + 1) * (y2 - y1 + 1)  
    idxs = np.argsort(y2)
    while len(idxs) > 0:  
        last = len(idxs) - 1  
        i = idxs[last]  
        pick.append(i)  
        suppress = [last]  
        for pos in range(0, last):  
            j = idxs[pos]  
            xx1 = max(x1[i], x1[j])  
            yy1 = max(y1[i], y1[j])  
            xx2 = min(x2[i], x2[j])  
            yy2 = min(y2[i], y2[j])  
            w = max(0, xx2 - xx1 + 1)  
            h = max(0, yy2 - yy1 + 1)  
            overlap = float(w * h) / area[j]  
            if overlap > overlapThresh:  
                suppress.append(pos)  
        idxs = np.delete(idxs, suppress)  
    return boxes[pick]  
    
#获取最终ROI的坐标    
def getPositions(img,propotion,shapes,origin_copy):    
    ret,thresh = cv2.threshold(img,127,255,0)
    tmp,contours,hierarchy=cv2.findContours(thresh,1,2)    
    hsvList=[]
    area_list=[]
#有效交通灯的比例    
    if contours :
        for i in range(len(contours)):
            if cv2.contourArea(contours[i])/propotion >0.0002 and cv2.contourArea(contours[i])/propotion < 0.1:
# 最小面积矩形                
                x,y,w,h=cv2.boundingRect(contours[i])
                
                y0=int(y-0.2*h)
                y1=int(y+0.9*h)
                x0=int(x-0.1*w)
                x1=int(x+w)
                if y0 <0:
                    y0=0
                if y1>shapes[0]:
                    y1=shapes[0]
                if x0<0:
                    x0=0
                if x1>shapes[1]:
                    x1=shapes[1]                
                area_list.append([x0,y0,x1,y1])
        area_list=np.array(area_list)         
        
# 非极大值抑制   
        prop=non_max_suppression_slow(area_list,0.7)
        if len(prop):
            i=0
            for (startX, startY, endX, endY) in prop:                 
#算出每个最小区域的亮度，进行从大到小的排序，将最亮的排在第一位
                HSV=origin_copy[startY:endY,startX:endX,:]
                HSV = cv2.cvtColor(HSV, cv2.COLOR_BGR2HSV)        
                H, S, V = cv2.split(HSV)
                v = np.mean(V)
                hsvList.append([i,v,[y0,y1,x0,x1]])
#                 hsvList.append([i,v,[y0,y1,x0,x1]])
#                 形状模板匹配                
                cv2.rectangle(origin_copy, (startX, startY), (endX, endY), (0, 255, 0), 2) 
                i+=1 
            hsvList=sorted(hsvList,key=lambda x : (x[1]),reverse=True) 
            order=hsvList[0][0]
            target=order
            ans=getShapes(origin_copy)                
            for i in range(len(ans)):
                template_name=ans[i][0]
                pos=ans[i][1]
#                 print('外部',hsvList[0][2])
#                 print('pos',pos)
                if(overlap([hsvList[0][2][2],hsvList[0][2][1],hsvList[0][2][3],hsvList[0][2][0]],pos)):
                    print('内部',hsvList[0][2])
                    if(template_name=='test.png'):
                        print('圆形')
                    elif(template_name=='test2.png'):
                        print("T型")
                    elif(template_name=='test3.png'):
                        print("左箭头")
                    elif(template_name=='test4.png'):
                        print("上箭头")
                    else:
                        print('右箭头')
                    cv2.rectangle(origin_copy,(pos[0],pos[1]), (pos[2],pos[3]), 255, 2)
#             cv2.putText(origin_copy, "IoU", (target[0], target[1]),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.imshow('res',origin_copy)
            return hsvList[0][2]
        else:
            return 0
    else:
        return 0
    

def main(path):
    start=time.clock()
    hsv,propotion,shapes,origin_copy=process(path)    
    colors=[]
    colors.append(getGreen(hsv))
    colors.append(getRed(hsv))
    colors.append(getYellow(hsv))
    # 节省时间复杂度

    areas=[]
    sumResult=0
    for i in range(len(colors)):
        closed = deNoise(list(colors[i].values())[0])
        coordinate = getPositions(closed,propotion,shapes,origin_copy)
        if isinstance(coordinate,list):
            print('coordinate',coordinate)
            HSV=origin_copy[coordinate[0]:coordinate[1],coordinate[2]:coordinate[3],:]
            HSV = cv2.cvtColor(HSV, cv2.COLOR_BGR2HSV)        
            H, S, V = cv2.split(HSV)
            v = np.mean(V)
            areas.append((list(colors[i].keys())[0],v))
        else:
            areas.append((list(colors[i].keys())[0],0))

    # 按颜色值的大小排序
    areas=sorted(areas,key=lambda x : (x[1]),reverse=True)
    print(areas)
    ended=time.clock()
    print('total:',ended-start)
    if int(areas[0][1]) != 0:
        print('The current color is:',areas[0][0])
    else:
        print('没有交通灯')
    cv2.waitKey(0)
    cv2.destroyAllWindows()


if __name__=='__main__':
    path="left0003.jpg"
    main(path)


In [39]:
d = {'name':'haohao','age':'4'}
key = list(d)[0]
print(key)
value = list(d.values())[0]
print(value)

name
haohao
